In this notebook, I will analyze NBM data and answer the following questions: 
1. How many unique BSLs are there in the latest version of each major release in Virginia?
2. How many ISPs provide service in Tennessee?
3. How many unserved BSLs are in Virginia in the latest version of each major release? An “unserved” BSL is one that does not have service exceeding 25Mbps download speed and 3Mbps upload speed from any ISP, ignoring ISPs that provide service to the BSL using any form of satellite Internet or *unlicensed* fixed wireless technology. They must also have either residential or “both” service and the service must be classified as “low latency”. Hint: you’ll want to filter based on technology code, download and upload speeds, business/residential code (values R and X), as well as the low latency flag.

In [6]:
# Import depednecies and read files. 
from get_parquet  import *
from categorize_bsl import *
from get_df import *
import concurrent.futures
import numpy as np
import pandas as pd
from libnbm import *
import duckdb
DATA_ROOT = '/playpen/data/nbm'
DATA_ROOT_SLICE = "/playpen/data/nbm/preprocessed"

Question 1: How many unique BSLs are there in the latest version of each major release in Virginia?

In [7]:
majors = [20220630, 20221231, 20230630, 20231231, 20240630]
minors = [20240510, 20241104, 20250210, 20250201, 20250218]
major_minor_dict = dict(zip(majors, minors))

# def return_df(major, minor, fips_code):
#     DATA_BASEDIR = '/home/playpen/data/nbm_evolution/data/nbm/bdc_single_file'
#     columns_to_read = ["location_id", "max_advertised_download_speed", 
#                        "max_advertised_upload_speed", "technology", 
#                        "low_latency", "business_residential_code"]
#     table = pyarrow_pq.read_pandas(f"{DATA_BASEDIR}/{major}/{minor}/bdc_{str(fips_code).zfill(2)}_single_nbm.parquet", columns = columns_to_read, use_threads=True)
#     return table.to_pandas()

def return_df(major, minor, fips_code):
    DATA_BASEDIR = '/home/playpen/data/nbm_evolution/data/nbm/bdc_single_file'
    table = pyarrow_pq.read_pandas(f"{DATA_BASEDIR}/{major}/{minor}/bdc_{str(fips_code).zfill(2)}_single_nbm.parquet", use_threads=True)
    return table.to_pandas()

def get_bsl_count(major):
    df = return_df(major, major_minor_dict[major], 51)
    unique_locs_count = df['location_id'].nunique()
    return(f"For major release {major} and minor release {major_minor_dict[major]}, there are {unique_locs_count} unique BSLs")

result = []
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    result = executor.map(get_bsl_count, majors)

print("Here are the number of unique BSLs in the latest version of each major release in Virginia: \n")
for r in result: 
    print(r)

Here are the number of unique BSLs in the latest version of each major release in Virginia: 



FileNotFoundError: /home/playpen/data/nbm_evolution/data/nbm/bdc_single_file/20220630/20240510/bdc_51_single_nbm.parquet

DUCK DB QUESTION 1

In [ ]:
majors = [20220630, 20221231, 20230630, 20231231, 20240630]
minors = [20240510, 20241104, 20250210, 20250201, 20250304]
major_minor_dict = dict(zip(majors, minors))

def get_bsl_count(major):
    minor = major_minor_dict[major]
    fips_code = 51  # Virginia
    file_path = os.path.join(DATA_ROOT_SLICE, f"nbm_{str(major)}_{str(minor)}_zstd.parquet")
    
    # Query to count unique location_ids
    query = f"""
    SELECT
        COUNT(DISTINCT location_id) AS unique_location_ids
    FROM '{file_path}'
    WHERE state_fips = {fips_code}
    """
    result = duckdb.query(query).fetchall()
    unique_location_ids = result[0][0]
    
    return f"For major release {major} and minor release {minor}, there are {unique_location_ids} unique BSLs"


result = []
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    result = executor.map(get_bsl_count, majors)

print("Here are the number of unique BSLs in the latest version of each major release in Virginia: \n")
for r in result: 
    print(r)

Here are the number of unique BSLs in the latest version of each major release in Virginia: 

For major release 20220630 and minor release 20240510, there are 2840311 unique BSLs
For major release 20221231 and minor release 20241104, there are 2914429 unique BSLs
For major release 20230630 and minor release 20250210, there are 2920197 unique BSLs
For major release 20231231 and minor release 20250201, there are 2907830 unique BSLs
For major release 20240630 and minor release 20250304, there are 2911038 unique BSLs


Question 2: How many ISPs provide service in Tennessee?

In [ ]:
majors = [20220630, 20221231, 20230630, 20231231, 20240630]
minors = [20240510, 20241104, 20250210, 20250201, 20250304]
major_minor_dict = dict(zip(majors, minors))

def count_brand_name(major):
    df = parquet_to_df(major, major_minor_dict[major], 47)
    return f"For major release {major} and minor release {major_minor_dict[major]}, there are {df['provider_id'].nunique()} unique ISPs"


with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    result = executor.map(count_brand_name, majors)

print("Here are the number of unique ISPs in the latest version of each major release in Tennessee: \n")

for r in result: 
    print(r)

Here are the number of unique ISPs in the latest version of each major release in Tennessee: 

For major release 20220630 and minor release 20240510, there are 96 unique ISPs
For major release 20221231 and minor release 20241104, there are 98 unique ISPs
For major release 20230630 and minor release 20250210, there are 94 unique ISPs
For major release 20231231 and minor release 20250201, there are 99 unique ISPs
For major release 20240630 and minor release 20250304, there are 102 unique ISPs


DUCK DB QUESTION 2

In [ ]:
majors = [20220630, 20221231, 20230630, 20231231, 20240630]
minors = [20240510, 20241104, 20250210, 20250201, 20250304]
major_minor_dict = dict(zip(majors, minors))

def count_brand_name(major):
    minor = major_minor_dict[major]
    file_path = os.path.join(DATA_ROOT_SLICE, f"nbm_{str(major)}_{str(minor)}_zstd.parquet")

    query = f"""
    SELECT COUNT(DISTINCT provider_id) AS unique_isps
    FROM '{file_path}'
    WHERE state_fips = 47
    """
    
    result = duckdb.query(query).fetchall()
    unique_isps = result[0][0]
    
    return f"For major release {major} and minor release {minor}, there are {unique_isps} unique ISPs in Tennessee."

# Run in parallel
result = []
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    result = executor.map(count_brand_name, majors)

print("Here are the number of unique ISPs in the latest version of each major release in Tennessee:\n")

for r in result: 
    print(r)

Here are the number of unique ISPs in the latest version of each major release in Tennessee:

For major release 20220630 and minor release 20240510, there are 96 unique ISPs in Tennessee.
For major release 20221231 and minor release 20241104, there are 98 unique ISPs in Tennessee.
For major release 20230630 and minor release 20250210, there are 94 unique ISPs in Tennessee.
For major release 20231231 and minor release 20250201, there are 99 unique ISPs in Tennessee.
For major release 20240630 and minor release 20250304, there are 102 unique ISPs in Tennessee.


Question 3: How many unserved BSLs are in Virginia in the latest version of each major release? An “unserved” BSL is one that does not have service exceeding 25Mbps download speed and 3Mbps upload speed from any ISP, ignoring ISPs that provide service to the BSL using any form of satellite Internet or unlicensed fixed wireless technology. They must also have either residential or “both” service and the service must be classified as “low latency”. Hint: you’ll want to filter based on technology code, download and upload speeds, business/residential code (values R and X), as well as the low latency flag.

In [ ]:
# Define directory and mappings
DATA_BASEDIR = "/home/playpen/data/nbm_evolution/data/nbm/bdc_single_file"
majors = [20220630, 20221231, 20230630, 20231231, 20240630]
minors = [20240510, 20241104, 20250210, 20250201, 20250218]
major_minor_dict = dict(zip(majors, minors))

technology_codes = {
    10: "Copper Wire",
    40: "Coaxial Cable / HFC",
    50: "Optical Carrier / Fiber to the Premises",
    60: "Geostationary Satellite",
    61: "Non-geostationary Satellite",
    70: "Unlicensed Terrestrial Fixed Wireless",
    71: "Licensed Terrestrial Fixed Wireless",
    72: "Licensed-by-Rule Terrestrial Fixed Wireless",
    0: "Other"
}

valid_technologies = [50, 40, 10, 71]
valid_res_codes = ["X", "R"]

def unserved_per_major(major):
    df = parquet_to_df(major, major_minor_dict[major], 51)
    new_df = get_best_status(df)
    unserved_count = new_df['status'].value_counts().get('unserved', 0)
    return (f"For major release {major} and minor release {major_minor_dict[major]}, there are {unserved_count} unserved BSLs")

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    result = executor.map(unserved_per_major, majors)

print("Here are the number of unserved BSLs in the latest version of each major release in Virginia: \n")


Here are the number of unserved BSLs in the latest version of each major release in Virginia: 

For major release 20220630 and minor release 20240510, there are 367789 unserved BSLs
For major release 20221231 and minor release 20241104, there are 373066 unserved BSLs
For major release 20230630 and minor release 20250210, there are 368998 unserved BSLs
For major release 20231231 and minor release 20250201, there are 305541 unserved BSLs
For major release 20240630 and minor release 20250218, there are 276281 unserved BSLs


In [15]:

majors = [20220630, 20221231, 20230630, 20231231, 20240630]
minors = [20240510, 20241104, 20250210, 20250201, 20250218]
major_minor_dict = dict(zip(majors, minors))
DATA_ROOT_SLICE = "/playpen/data/nbm/preprocessed"


technology_codes = {
    10: "Copper Wire",
    40: "Coaxial Cable / HFC",
    50: "Optical Carrier / Fiber to the Premises",
    60: "Geostationary Satellite",
    61: "Non-geostationary Satellite",
    70: "Unlicensed Terrestrial Fixed Wireless",
    71: "Licensed Terrestrial Fixed Wireless",
    72: "Licensed-by-Rule Terrestrial Fixed Wireless",
    0: "Other"
}

valid_technologies = [50, 40, 10, 71, 72]
invalid_technologies = [60, 61, 0]
valid_res_codes = ["X", "R"]

def unserved_per_major(major):
    minor = major_minor_dict[major]
    file_path = os.path.join(DATA_ROOT_SLICE, f"nbm_{str(major)}_{str(minor)}_zstd.parquet")
    
    query = f"""
    SELECT COUNT(*) AS unserved_count
    FROM (
        SELECT location_id,
            MAX(
                CASE
                    WHEN max_advertised_download_speed > 100 AND max_advertised_upload_speed > 20 THEN 2
                    WHEN max_advertised_download_speed > 25 AND max_advertised_upload_speed > 3 THEN 1
                    ELSE 0
                END
            ) AS status
        FROM '{file_path}'
        WHERE state_fips = 51
          AND technology IN ({','.join(map(str, valid_technologies))})
          AND business_residential_code IN ({','.join(f"'{r}'" for r in valid_res_codes)})
          AND low_latency = 1
        GROUP BY location_id
    )
    WHERE status = 0  -- 0 = unserved
    """
    result = duckdb.query(query).fetchall()
    unserved_count = result[0][0]  
    return f"For major release {major} and minor release {major_minor_dict[major]}, there are {unserved_count} unserved BSLs"


# Run in parallel
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    result = executor.map(unserved_per_major, majors)

print("Here are the number of unserved BSLs in the latest version of each major release in Virginia:\n")

for r in result:
    print(r)


Here are the number of unserved BSLs in the latest version of each major release in Virginia:

For major release 20220630 and minor release 20240510, there are 340940 unserved BSLs
For major release 20221231 and minor release 20241104, there are 290628 unserved BSLs
For major release 20230630 and minor release 20250210, there are 254064 unserved BSLs
For major release 20231231 and minor release 20250201, there are 217991 unserved BSLs
For major release 20240630 and minor release 20250218, there are 158022 unserved BSLs
